# External Database Integration with Strands

작업중입니다. (시나리오에 따라 바뀌어야하며, 작업전 postgre sql조회해 실제 테이블에 어떻게 들어가있는지 조회하는 과정이 추가되어야합니다.

이 노트북에서는 Strands Agents 프레임워크를 사용하여 사내 데이터베이스가 PostgreSQL임을 가정하고, 사내 데이터베이스와 연동하는 에이전트를 만듭니다.

#### Install Strands agents and required dependencies

In [ ]:
%pip install strands-agents strands-agents-tools psycopg2-binary --quiet

#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.37.1** or higher.

In [ ]:
%pip show boto3

#### Import required libraries

In [ ]:
import boto3
import json
import psycopg2
from typing import Dict, Any
from strands import Agent, tool
from strands.models import BedrockModel
from collections import defaultdict

# Get AWS account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

print(f"Region: {region}")
print(f"Account ID: {account_id}")

#### Database Configuration
PostgreSQL 데이터베이스 연결 정보를 설정합니다.

In [ ]:
# Database configuration - Update these values with your actual RDS endpoint
DB_CONFIG = {
    "host": "YOUR_RDS_ENDPOINT",  # 워크샵 메인 output 정보에 있으며 가이드에 따라 입력합니다.
    "port": 5432,
    "database": "workshopdb",
    "user": "dbadmin",
    "password": "postgres"
}

print("Database configuration loaded")
print(f"Host: {DB_CONFIG['host']}")
print(f"Database: {DB_CONFIG['database']}")

#### Define Database Tools
데이터베이스 스키마 조회와 쿼리 실행을 위한 도구들을 정의합니다.

In [ ]:
@tool
async def fetch_table_schema() -> dict:
    """데이터베이스 쿼리 관련 작업을 할 때 항상 먼저 실행해야 하는 도구입니다. public 스키마의 테이블과 컬럼 정보를 조회합니다."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT
                    c.relname AS table_name,
                    a.attname AS column_name,
                    pg_catalog.format_type(a.atttypid, a.atttypmod) AS data_type,
                    d.description AS column_comment
                FROM pg_catalog.pg_attribute a
                JOIN pg_catalog.pg_class c ON a.attrelid = c.oid
                JOIN pg_catalog.pg_namespace n ON c.relnamespace = n.oid
                LEFT JOIN pg_catalog.pg_description d ON a.attrelid = d.objoid AND a.attnum = d.objsubid
                WHERE a.attnum > 0 AND NOT a.attisdropped
                AND n.nspname = 'public'
                AND c.relkind = 'r'
                ORDER BY c.relname, a.attnum;
            """)
            
            rows = cursor.fetchall()
            # 테이블별로 그룹화
            tables = defaultdict(list)
            for table_name, column_name, data_type, column_comment in rows:
                tables[table_name].append({
                    "column_name": column_name,
                    "data_type": data_type,
                    "comment": column_comment
                })
            
            result = [{"table_name": tbl, "columns": cols} for tbl, cols in tables.items()]
            return {"message": result, "status": "success"}

    except Exception as e:
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

@tool
async def execute_postgres_query(query: str) -> dict:
    """적합한 도구를 찾지 못한다면 항상 도구(this tool : database query)를 통해 데이터를 조회합니다."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        cur.execute(query)
        result = cur.fetchall()
        conn.commit()
        return {
            "message": "\n".join(str(row) for row in result),
            "status": "success"
        }
    except Exception as e:
        return {
            "message": str(e)
        }
    finally:
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()

print("Database tools defined successfully:")
print(f"- fetch_table_schema: {fetch_table_schema.__doc__.split('.')[0].strip()}")
print(f"- execute_postgres_query: {execute_postgres_query.__doc__.split('.')[0].strip()}")

#### Create the Database Agent
PostgreSQL 데이터베이스와 연동하는 Strands 에이전트를 생성합니다.

In [ ]:
# Agent configuration
db_agent_instruction = """
당신은 의료 데이터베이스 전문가 AI입니다. PostgreSQL 데이터베이스에 저장된 임상 및 유전체 데이터를 분석하고 쿼리할 수 있습니다.

주요 기능:
1. 데이터베이스 스키마 탐색 및 이해
2. 자연어 질문을 SQL 쿼리로 변환
3. 임상 데이터 분석 및 통계 제공
4. 유전체 데이터 패턴 분석

데이터베이스 정보:
- chemotherapy_survival: 화학요법 후 환자 생존 데이터
  - LRIG1: 유전자 발현량
  - Survival_Status: 생존 상태 (true/false)
  - Survival_Duration: 생존 기간
  - ExpressionGroup: 발현 그룹 (0/1)

- clinical_genomic: 임상 및 유전체 정보
  - 다양한 유전자 발현량 컬럼들
  - 환자 임상 정보 (나이, 성별, 흡연상태 등)
  - 종양 정보 (위치, 병리학적 병기 등)
  - 치료 정보 (화학요법, 방사선치료 등)

항상 도움이 되고 정확한 정보를 제공하며, 의료 데이터의 중요성을 인식하고 신중하게 분석하세요.
"""

# Create the Bedrock model
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    client=bedrock_client
)

# Create the agent
db_agent = Agent(
    system_prompt=db_agent_instruction,
    model=model,
    tools=[fetch_table_schema, execute_postgres_query]
)

print("Database Agent created successfully with Strands framework")

# Test the Database Agent
데이터베이스 에이전트를 다양한 쿼리로 테스트해봅니다.

In [ ]:
# Enable detailed logging for debugging
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

#### Test 1: Database Connection and Schema

In [ ]:
# Test database connection and schema exploration
query = "데이터베이스에 어떤 테이블들이 있는지 알려주세요"

print("Query:", query)
print("\nResponse:")
try:
    response = db_agent(query)
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

#### Test 2: Simple Database Query

In [ ]:
# Test simple database query
query = "현재 데이터베이스 시간을 알려주세요"

print("Query:", query)
print("\nResponse:")
response = db_agent(query)

#### Test 3: Clinical Data Analysis

In [ ]:
# Test clinical data analysis
query = "선암(Adenocarcinoma) 진단을 받은 환자의 평균 연령은 얼마입니까?"

print("Query:", query)
print("\nResponse:")
response = db_agent(query)

#### Test 4: Complex Analysis Query

In [ ]:
# Test complex analysis
query = "50세 이상 진단을 받은 환자의 수는 몇 명이며, 흡연 여부는 어떻게 됩니까?"

print("Query:", query)
print("\nResponse:")
response = db_agent(query)

#### Test 5: Survival Analysis

In [ ]:
# Test survival analysis
query = "화학요법 후 생존한 환자와 사망한 환자의 LRIG1 유전자 발현량 평균을 비교해주세요"

print("Query:", query)
print("\nResponse:")
response = db_agent(query)

#### Test 6: Mutation Analysis

In [ ]:
# Test mutation analysis
query = "EGFR 돌연변이를 가진 환자들의 비율과 이들의 생존율을 분석해주세요"

print("Query:", query)
print("\nResponse:")
response = db_agent(query)

# Database Schema Information

## chemotherapy_survival 테이블
화학요법(항암치료) 후 환자들의 생존상태를 담고 있는 테이블

**컬럼 설명:**
- `lrig1`: Leucine-rich repeats and immunoglobulin-like domains 1 유전자 발현량
  - 세포 성장 조절 및 종양 억제 유전자로 작용할 수 있음
  - 항암 치료 반응성 또는 예후 지표로서 활용
  - LRIG1의 발현수준이 생존율과 상관관계를 가짐
- `survival_status`: 생존 상태 - true/false
- `survival_duration`: 생존 기간
- `expressiongroup`: 발현수준에 따라 나눈 그룹 - 0/1

## clinical_genomic 테이블
암 환자의 임상 및 유전체 정보를 포함하고 있는 테이블

**주요 컬럼들:**
- 다양한 유전자들의 발현량 컬럼
- `patient_affiliation`: 병원, 연구기관 등의 환자 소속정보
- `age_at_histological_diagnosis`: 조직학적 진단 당시 나이
- `weight_lbs`: 체중
- `gender`: 성별
- `ethnicity`: 인종
- `smoking_status`: 흡연 상태
- `pack_years`: 누적 흡연량
- `percent_gg`: 종양 내 GGO 비율
- `tumor_location_*`: 종양 위치
- `histology`: 조직학적 암 유형
- `pathological_*`: 병리적 병기/결과
- `histopathological_grade`: 암세포 분화 정도
- `lymphovascular_invasion`: 림프관 또는 혈관 침습 여부
- `pleural_invasion`: 흉막 침습 여부
- `egfr_mutation_status`: EGFR 유전자 돌연변이 상태
- `kras_mutation_status`: KRAS 유전자 돌연변이 상태
- `alk_translocation_status`: ALK 유전자 전좌 여부
- `adjuvant_treatment`: 보조 치료 여부
- `chemotherapy`: 항암화학요법 수행 여부
- `radiation`: 방사선 치료 여부
- `recurrence`: 암 재발 여부
- `recurrence_location`: 암 재발 부위
- `survival_status`: 생존 상태
- `time_to_death`: 진단 후 사망까지의 시간
- `days_between_ct_and_surgery`: CT 촬영일과 수술일 사이의 간격
- `survival_duration`: 전체 생존 기간

# Conclusion

이 노트북에서는 Strands Agents 프레임워크를 사용하여 PostgreSQL 데이터베이스와 연동하는 에이전트를 성공적으로 구현했습니다.

## 주요 구현 내용:

### 1. 데이터베이스 도구 구현
- **fetch_table_schema**: 데이터베이스 스키마 탐색 도구
- **execute_postgres_query**: SQL 쿼리 실행 도구

### 2. 에이전트 기능
- 자연어 질문을 SQL 쿼리로 변환
- 임상 및 유전체 데이터 분석
- 생존 분석 및 돌연변이 분석
- 통계적 분석 및 패턴 발견

### 3. 데이터셋 특징
- **chemotherapy_survival**: 화학요법 후 환자 생존 데이터
- **clinical_genomic**: 초기 단계 비소세포폐암 환자 코호트 데이터
- 종양 조직의 유전자 발현 데이터
- 환자의 임상 및 인구통계학적 정보

## Strands 프레임워크의 장점:

1. **간편한 도구 정의**: `@tool` 데코레이터를 사용한 직관적인 도구 구현
2. **직접적인 통합**: Lambda 함수 없이 직접 데이터베이스 연결
3. **유연한 에이전트 구성**: 시스템 프롬프트와 도구의 조합으로 전문화된 에이전트 생성
4. **쉬운 테스트**: 노트북 환경에서 직접 테스트 가능
5. **비용 효율성**: Lambda 함수 비용 제거
6. **빠른 실행**: Lambda 콜드 스타트 지연 제거

## 사용 예시:
- 선암종 환자들의 평균 연령 조회
- 편평세포암종 환자들의 가장 흔한 병리학적 T 병기 분석
- EGFR 돌연변이를 가진 환자들의 비율 계산
- 화학요법 후 생존율과 유전자 발현량 상관관계 분석

이 구현을 통해 의료 연구자들은 복잡한 SQL을 작성하지 않고도 자연어로 데이터베이스를 쿼리하고 분석할 수 있게 되었습니다.